In [6]:
pip install sqlglot

   ---------------------------------------- 0.0/403.1 kB ? eta -:--:--
   ----- ---------------------------------- 51.2/403.1 kB 1.3 MB/s eta 0:00:01
   ----------- ---------------------------- 112.6/403.1 kB 1.7 MB/s eta 0:00:01
   ------------------ --------------------- 184.3/403.1 kB 1.9 MB/s eta 0:00:01
   ------------------ --------------------- 184.3/403.1 kB 1.9 MB/s eta 0:00:01
   --------------------- ------------------ 215.0/403.1 kB 1.0 MB/s eta 0:00:01
   ---------------------------------------- 403.1/403.1 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [265]:
import sqlparse
from sqlparse.sql import IdentifierList, Identifier, Parenthesis, Function, Operation
from sqlparse.tokens import Keyword, DML
import pandas as pd

def is_subselect(parsed):
    """Check if the parsed token is a subselect."""
    if not parsed.is_group:
        return False
    for item in parsed.tokens:
        if item.ttype is DML and item.value.upper() == 'SELECT':
            return True
    return False

def extract_from_part(parsed):
    """Extract the FROM part of the SQL query."""

   
    from_seen = False
    for item in parsed.tokens:
        if from_seen:
            if is_subselect(item):
                for x in extract_from_part(item):
                    yield x
                # print(1, ':', item, isinstance(item, Identifier))
            elif item.ttype is Keyword and item.value.upper() in ['JOIN', 'LEFT JOIN', 'RIGHT JOIN', 'INNER JOIN', 'CROSS JOIN']:
                yield item
                # print(2, ':', item, isinstance(item, Identifier))
            elif item.ttype is Keyword:
                # print(3, ':', item, isinstance(item, Identifier))
                return
            else:
                yield item
                # print(4, ':', item, isinstance(item, Identifier))
        elif item.ttype is Keyword and item.value.upper() == 'FROM':
            # print(5, ':', item, isinstance(item, Identifier))
            from_seen = True

def extract_tables_from_query(parsed):
    """Extract table names from the parsed SQL query."""
    tables = []
    for item in parsed.tokens:
        if isinstance(item, IdentifierList):
            for identifier in item.get_identifiers():
                tables.append(identifier.get_real_name())
        elif isinstance(item, Identifier):
            tables.append(item.get_real_name())
        elif isinstance(item, Parenthesis):
            # Handle nested queries
            tables.extend(extract_tables_from_query(item))
    return tables

def extract_tables(sql):
    """Extract table names from the SQL query."""
    tables = []
    parsed = sqlparse.parse(sql)
    for statement in parsed:
        for item in extract_from_part(statement):
            if isinstance(item, IdentifierList):
                for identifier in item.get_identifiers():
                    tables.append(identifier.get_real_name())
            elif isinstance(item, Identifier):
                tables.append(item.get_real_name())
            elif isinstance(item, Parenthesis):
                # Handle nested queries
                tables.extend(extract_tables_from_query(item))
    return tables

def extract_tables_with_aliases(sql):
    tables = []
    parsed = sqlparse.parse(sql)
    stmt = parsed[0]
    
    for item in extract_from_part(stmt):
        if isinstance(item, IdentifierList):
            for identifier in item.get_identifiers():
                tables.append((identifier.get_real_name(), identifier.get_alias() or identifier.get_real_name()))
        elif isinstance(item, Identifier):
            tables.append((item.get_real_name(), item.get_alias() or item.get_real_name()))

    return tables

import re

def extract_alias_column_pairs(sql):
    # Regular expression to match alias.column patterns
    pattern = re.compile(r'(\b\w+\b)\.(\b\w+\b)', re.IGNORECASE)
    columns = []
    tuples = []
    # Find all matches in the SQL statement
    matches = pattern.findall(sql)
    
    # Print the extracted alias.column pairs
    if matches:
        # print("Alias.Column Pairs:")
        for alias, column in matches:
            # print(f"- {alias}.{column}")
            columns.append(alias + "." + column)
            tuples = [tuple(col.split('.', 1)) for col in columns]
    else:
        print("No alias.column pairs found.")
    return tuples

# Example usage
sql = """
SELECT RI_X_CL.*
		,CV.CL_NM
	FROM RI_X_CL_RLTNP RI_X_CL
	JOIN CV ON RI_X_CL.CL_ID = CV.CL_ID
		AND RI_X_CL.EFF_DT <= #pDate#
		AND RI_X_CL.END_DT > #pDate#
	WHERE RI_X_CL.RI_X_CL_RLTNP_TP_ID = '1AD160750012C416E42B551CC5CA4A5F'
"""
formatted_sql = sqlparse.format(sql);
tables = extract_tables_with_aliases(formatted_sql);

df1 = pd.DataFrame(tables, columns=['Tables', 'Ailases'])
# print(tables)

print(df1)

columns = extract_alias_column_pairs(formatted_sql)

df2 = pd.DataFrame(columns, columns=['Ailases', 'Columns'])
# print(df2)

inner_join = pd.merge(df1, df2, on='Ailases', how='inner')

print(inner_join)

          Tables  Ailases
0  RI_X_CL_RLTNP  RI_X_CL
1             CV       CV
          Tables  Ailases              Columns
0  RI_X_CL_RLTNP  RI_X_CL                CL_ID
1  RI_X_CL_RLTNP  RI_X_CL               EFF_DT
2  RI_X_CL_RLTNP  RI_X_CL               END_DT
3  RI_X_CL_RLTNP  RI_X_CL  RI_X_CL_RLTNP_TP_ID
4             CV       CV                CL_NM
5             CV       CV                CL_ID


ModuleNotFoundError: No module named 'ply'

In [143]:
import sqlparse
from sqlparse.sql import IdentifierList, Identifier, Parenthesis
from sqlparse.tokens import Keyword, DML

def is_subselect(parsed):
    """Check if the parsed token is a subselect."""
    if not parsed.is_group:
        return False
    for item in parsed.tokens:
        if item.ttype is DML and item.value.upper() == 'SELECT':
            return True
    return False

def extract_from_part(parsed):
    """Extract the FROM part of the SQL query."""
    from_seen = False
    for item in parsed.tokens:
        if from_seen:
            if is_subselect(item):
                for x in extract_from_part(item):
                    yield x
            elif item.ttype is Keyword and item.value.upper() in ['JOIN', 'LEFT JOIN', 'RIGHT JOIN', 'INNER JOIN']:
                yield item
            elif item.ttype is Keyword:
                return
            else:
                yield item
        elif item.ttype is Keyword and item.value.upper() == 'FROM':
            from_seen = True

def extract_tables_from_query(parsed):
    """Extract table names from the parsed SQL query."""
    tables = []
    for item in parsed.tokens:
        if isinstance(item, IdentifierList):
            for identifier in item.get_identifiers():
                tables.append(identifier.get_real_name())
        elif isinstance(item, Identifier):
            tables.append(item.get_real_name())
        elif isinstance(item, Parenthesis):
            # Handle nested queries
            tables.extend(extract_tables_from_query(item))
    return tables

def extract_columns(sql):
    """Extract column names from the SQL query."""
    columns = []
    parsed = sqlparse.parse(sql)
    for statement in parsed:
        if statement.get_type() == 'SELECT':
            for token in statement.tokens:
                if isinstance(token, IdentifierList):
                    for identifier in token.get_identifiers():
                        columns.append(identifier.get_real_name())
                elif isinstance(token, Identifier):
                    columns.append(token.get_real_name())
                elif isinstance(token, Parenthesis):
                    # Handle nested queries
                    columns.extend(extract_columns(str(token)))
    return columns

def extract_tables(sql):
    """Extract table names from the SQL query."""
    tables = []
    parsed = sqlparse.parse(sql)
    for statement in parsed:
        for item in extract_from_part(statement):
            if isinstance(item, IdentifierList):
                for identifier in item.get_identifiers():
                    tables.append(identifier.get_real_name())
            elif isinstance(item, Identifier):
                tables.append(item.get_real_name())
            elif isinstance(item, Parenthesis):
                # Handle nested queries
                tables.extend(extract_tables_from_query(item))
    return tables

# Example usage
sql = """YOUR COMPLEX SQL QUERY HERE"""
tables = extract_tables(sql)
columns = extract_columns(sql)
print("Tables:", tables)
print("Columns:", columns)


SELECT LOAN.AC_AR_ID,
       SUM(TOT_AVL_LMT_AMT) TOT_AVL_LMT_AMT
FROM
  (SELECT AC_AR.AC_AR_ID ,
          nvl(RI_VAL.CLT_VALT - RI_VAL.LMT_CTB, 0) TOT_AVL_LMT_AMT
   FROM TWT_FNC_SVC_AR_ANL_FCT_FA_CHG AC_AR
   LEFT JOIN AR_X_RI_RLTNP AR_X_RI ON AR_X_RI.AR_ID = AC_AR.AC_AR_ID
   AND AR_X_RI.EFF_DT <= AC_AR.LAST_EFF_DT
   AND AR_X_RI.END_DT > AC_AR.LAST_EFF_DT
   AND AR_X_RI.AR_X_RI_RLTNP_TP_ID = 'B6C71A2D532689A626795C8D4B67A75D' -- LOAN_AR_X_COLATERAL

   AND AR_X_RI.SRC_STM_ID = '0A173E8726B6D97003B5EE634C0CCA92' -- FCC.CLTB_ACC_COLL_LINK_DTLS
LEFT JOIN RI ON AR_X_RI.RI_ID = RI.RI_ID
   AND RI.SRC_STM_ID = 'BB35A878C33FFBF19ADA056E5449DAAF' --FCC.GETM_COLLAT
LEFT JOIN
     (SELECT RI_VAL.RI_ID ,
             NVL(MAX(DECODE(CL_CODE, 'CLT_VALT', RI_VAL.VAL_AMT)), 0) CLT_VALT ,
             NVL(MAX(DECODE(CL_CODE, 'LMT_CTB', RI_VAL.VAL_AMT)), 0) LMT_CTB
      FROM RI_VAL
      JOIN CV ON RI_VAL.RI_VAL_TP_ID = CV.CL_ID
      AND CV.CL_SCM_CODE = 'RI_VAL_TP'
      AND CV.CL_CODE IN ('CL

In [173]:
import sqlparse
from sqlparse.sql import IdentifierList, Identifier
from sqlparse.tokens import Keyword, DML

def is_subselect(parsed):
    """Check if the parsed token is a subselect."""
    if not parsed.is_group:
        return False
    for item in parsed.tokens:
        if item.ttype is DML and item.value.upper() == 'SELECT':
            return True
    return False

def extract_from_part(parsed):
    """Extract the FROM part of the SQL query."""
    from_seen = False
    for item in parsed.tokens:
        if from_seen:
            if is_subselect(item):
                for x in extract_from_part(item):
                    yield x
            elif item.ttype is Keyword:
                return
            else:
                yield item
        elif item.ttype is Keyword and item.value.upper() == 'FROM':
            from_seen = True

def extract_tables(sql):
    """Extract table names from the SQL query."""
    tables = []
    parsed = sqlparse.parse(sql)
    for item in parsed:
        for x in extract_from_part(item):
            if isinstance(x, IdentifierList):
                for identifier in x.get_identifiers():
                    tables.append(identifier.get_real_name())
            elif isinstance(x, Identifier):
                tables.append(x.get_real_name())
    return tables

def extract_columns(sql):
    """Extract column names from the SQL query."""
    columns = []
    parsed = sqlparse.parse(sql)
    for statement in parsed:
        if statement.get_type() == 'SELECT':
            for token in statement.tokens:
                if isinstance(token, IdentifierList):
                    for identifier in token.get_identifiers():
                        columns.append(identifier.get_real_name())
                elif isinstance(token, Identifier):
                    columns.append(token.get_real_name())
    return columns


def extract_items(sql):
    i= 0
    """Extract table names from the SQL query."""
    items = []
    parsed = sqlparse.parse(sql)
    for item in parsed:
        print(item)
        i = i + 1
        print(i)
    

sql = """
SELECT RI_VAL.RI_ID
		,NVL(MAX(DECODE(CL_CODE, 'CLT_VALT', RI_VAL.VAL_AMT)), 0) CLT_VALT
		,NVL(MAX(DECODE(CL_CODE, 'LMT_CTB', RI_VAL.VAL_AMT)), 0) LMT_CTB
	FROM RI_VAL
	JOIN CV ON RI_VAL.RI_VAL_TP_ID = CV.CL_ID
		AND CV.CL_SCM_CODE = 'RI_VAL_TP'
		AND CV.CL_CODE IN (
			'CLT_VALT'
			,'LMT_CTB'
			)
	WHERE RI_VAL.EFF_DT <= #pDate#
		AND RI_VAL.END_DT > #pDate#
	GROUP BY RI_VAL.RI_ID
"""
formatted_sql = sqlparse.format(sql, reindent=True, keyword_case='upper')

extract_items(sql)

tables = extract_tables(sql)
columns = extract_columns(sql)

print("Tables:", tables)
print("Columns:", columns)



SELECT RI_VAL.RI_ID
		,NVL(MAX(DECODE(CL_CODE, 'CLT_VALT', RI_VAL.VAL_AMT)), 0) CLT_VALT
		,NVL(MAX(DECODE(CL_CODE, 'LMT_CTB', RI_VAL.VAL_AMT)), 0) LMT_CTB
	FROM RI_VAL
	JOIN CV ON RI_VAL.RI_VAL_TP_ID = CV.CL_ID
		AND CV.CL_SCM_CODE = 'RI_VAL_TP'
		AND CV.CL_CODE IN (
			'CLT_VALT'
			,'LMT_CTB'
			)
	WHERE RI_VAL.EFF_DT <= #pDate#
		AND RI_VAL.END_DT > #pDate#
	GROUP BY RI_VAL.RI_ID

1
Tables: ['RI_VAL']
Columns: ['RI_ID', 'NVL', 'NVL', 'RI_VAL', 'CV', 'CL_CODE', 'RI_ID']
